# MODULO 2: FINAL

**1)EXTRACCION DE DATOS DE PELICULAS**

In [251]:
import requests
import pandas as pd
import mysql.connector
from mysql.connector import Error
import numpy as np
import json

In [252]:
url = 'https://beta.adalab.es/resources/apis/pelis/pelis.json'
datos = requests.get(url)
print(datos.status_code)
print(datos.content)
datos_movies = datos.json()

200
b'[\n    {\n        "id": 1,\n        "titulo": "The Godfather",\n        "a\xc3\xb1o": 1972,\n        "duracion": 175,\n        "genero": "Crimen",\n        "adultos": false,\n        "subtitulos": [\n            "es",\n            "en"\n        ]\n    },\n    {\n        "id": 2,\n        "titulo": "The Godfather Part II",\n        "a\xc3\xb1o": 1974,\n        "duracion": 202,\n        "genero": "Crimen",\n        "adultos": false,\n        "subtitulos": [\n            "es",\n            "en"\n        ]\n    },\n    {\n        "id": 3,\n        "titulo": "Pulp Fiction",\n        "a\xc3\xb1o": 1994,\n        "duracion": 154,\n        "genero": "Crimen",\n        "adultos": true,\n        "subtitulos": [\n            "es",\n            "en"\n        ]\n    },\n    {\n        "id": 4,\n        "titulo": "Forrest Gump",\n        "a\xc3\xb1o": 1994,\n        "duracion": 142,\n        "genero": "Drama",\n        "adultos": false,\n        "subtitulos": [\n            "es",\n            

In [253]:
datos_movies

[{'id': 1,
  'titulo': 'The Godfather',
  'año': 1972,
  'duracion': 175,
  'genero': 'Crimen',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 2,
  'titulo': 'The Godfather Part II',
  'año': 1974,
  'duracion': 202,
  'genero': 'Crimen',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 3,
  'titulo': 'Pulp Fiction',
  'año': 1994,
  'duracion': 154,
  'genero': 'Crimen',
  'adultos': True,
  'subtitulos': ['es', 'en']},
 {'id': 4,
  'titulo': 'Forrest Gump',
  'año': 1994,
  'duracion': 142,
  'genero': 'Drama',
  'adultos': False,
  'subtitulos': ['es', 'en', 'fr']},
 {'id': 5,
  'titulo': 'The Dark Knight',
  'año': 2008,
  'duracion': 152,
  'genero': 'Acción',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 6,
  'titulo': 'Fight Club',
  'año': 1999,
  'duracion': 139,
  'genero': 'Drama',
  'adultos': True,
  'subtitulos': ['es', 'en']},
 {'id': 7,
  'titulo': 'Inception',
  'año': 2010,
  'duracion': 148,
  'genero': 'Ciencia ficción',
  'adultos

**CONECCION A MYSQL**

In [254]:

try:
    cnx = mysql.connector.connect(
    host="localhost",
    user="root",
    password="ruth",
    database="movies"
    )
    print("conexion exitosa")
except Error as e:
    print("Error al conectar a la base de datos", e)


conexion exitosa


**2)CREAR LA BASE DE DATOS**

In [255]:
try:
    mycursor = cnx.cursor()
    query = "CREATE DATABASE IF NOT EXISTS MOVIES"
    mycursor.execute(query)
    print("QUERY EXITOSA")
except:
    print("ERROR")

    



QUERY EXITOSA


In [256]:
mycursor.execute("USE MOVIES")
query="""CREATE TABLE IF NOT EXISTS movies (
        id INT AUTO_INCREMENT PRIMARY KEY,
        titulo VARCHAR(255),
        anio_lanzamiento INT,
        duracion_minutos INT,
        genero VARCHAR(100),
        contenido_adultos BOOLEAN,
        subtitulos VARCHAR(50)
    )"""
mycursor.execute(query)

**insertas en bd**

In [257]:
df_movies = pd.DataFrame(datos_movies)
df_movies

,id,titulo,año,duracion,genero,adultos,subtitulos
0,1,The Godfather,1972,175,Crimen,False,"[es, en]"
1,2,The Godfather Part II,1974,202,Crimen,False,"[es, en]"
2,3,Pulp Fiction,1994,154,Crimen,True,"[es, en]"
3,4,Forrest Gump,1994,142,Drama,False,"[es, en, fr]"
4,5,The Dark Knight,2008,152,Acción,False,"[es, en]"
...,...,...,...,...,...,...,...
95,96,La vita è bella,1997,116,Drama,False,"[es, en, it]"
96,97,Requiem for a Dream,2000,102,Drama,True,"[es, en]"
97,98,Memento,2000,113,Thriller,True,"[es, en]"
98,99,Eternal Sunshine of the Spotless Mind,2004,108,Drama,False,"[es, en]"


In [258]:
query_insert = """INSERT INTO movies (titulo, anio_lanzamiento, duracion_minutos, genero, contenido_adultos,subtitulos)
VALUES (%s, %s, %s, %s, %s, %s);"""


datos =  df_clean.values.tolist()




In [259]:
df_clean = df_movies.replace({np.nan: None,"nan":None, "NaN":None})

for col in df_movies.columns:
    df_movies[col] = df_movies[col].apply(
        lambda x: json.dumps(x) if isinstance(x, list) else x
    )

df_clean = df_movies.where(pd.notnull(df_movies), None)

df_clean = df_clean.drop(columns=["id"], errors="ignore")

print(df_clean.columns.tolist())
print(len(df_clean.columns)) # para poder matcher con los campos de la tabla ignore el id que es autoincrementaly esto me da 6 columnas que es lo que necesito


['titulo', 'año', 'duracion', 'genero', 'adultos', 'subtitulos']
6


In [260]:
mycursor.executemany(query_insert, datos)
cnx.commit()
print(f"{mycursor.rowcount} registros insertados exitosamente.")


100 registros insertados exitosamente.


# 4)OBTENCION DE INFORMACION A PARTIR DE LOS DATOS

**PELICULAS CON DURACION SUPERIOR A 120 min**

In [261]:
query = "SELECT COUNT(*) FROM movies WHERE duracion_minutos > 120"
mycursor.execute(query)

# Obtener el resultado
resultado = mycursor.fetchone()
print(f"Número de películas con duración superior a 120 minutos: {resultado[0]}")

Número de películas con duración superior a 120 minutos: 885


**PELICULAS QUE INCLUYEN SUBTITULOS EN ESPANOL**

In [262]:
query = "SELECT COUNT(*) FROM movies WHERE SUBTITULOS LIKE '%es%'" # LIKE '%es%' detecta cualquier fila donde la columna SUBTITULOS contenga la cadena "es" en algún lugar.
mycursor.execute(query)
# Obtener el resultado
resultado = mycursor.fetchone()
print(f"Número de películas con subtítulos en español: {resultado[0]}")

Número de películas con subtítulos en español: 1500


**PELICULAS CON CONTENIDO PARA ADULTO**

In [263]:
query = "SELECT * FROM movies WHERE contenido_adultos = TRUE"
mycursor.execute(query)
resultado = mycursor.fetchall()
print(f"Número de películas con contenido para adultos: {len(resultado)}")

Número de películas con contenido para adultos: 705


**PELICULA MAS ANTIGUA REGISTRADA EN LA BD**

In [264]:
query = "SELECT titulo, anio_lanzamiento FROM movies ORDER BY anio_lanzamiento ASC LIMIT 1"
mycursor.execute(query)

# fetchone() trae solo la primera fila
pelicula = mycursor.fetchone()
print(f"Título: {pelicula[0]}, Año de lanzamiento: {pelicula[1]}")

Título: Citizen Kane, Año de lanzamiento: 1941


**EL PROMEDIO DE DURACION DE LAS PELICULAS AGRUPADAS POR GENERO**

In [265]:
query = "SELECT genero, AVG(duracion_minutos) as duracion_media FROM movies GROUP BY genero"
mycursor.execute(query)
resultados = mycursor.fetchall()
for genero, duracion_media in resultados:
    print(f"Género: {genero}, Duración media: {duracion_media:.2f} minutos")


Género: Crimen, Duración media: 154.29 minutos
Género: Drama, Duración media: 126.26 minutos
Género: Acción, Duración media: 139.44 minutos
Género: Ciencia ficción, Duración media: 136.31 minutos
Género: Romance, Duración media: 139.67 minutos
Género: Bélico, Duración media: 161.00 minutos
Género: Thriller, Duración media: 121.67 minutos
Género: Musical, Duración media: 128.00 minutos
Género: Fantasía, Duración media: 159.80 minutos
Género: Aventura, Duración media: 133.00 minutos
Género: Animación, Duración media: 103.00 minutos
Género: Biografía, Duración media: 142.33 minutos
Género: Terror, Duración media: 116.57 minutos
Género: Suspense, Duración media: 120.00 minutos
Género: Comedia, Duración media: 117.00 minutos
Género: Western, Duración media: 166.50 minutos


** REGISTRO DE PELICULAS POR ANIOS EN LA BD ORDENADAS DE MAYOR A MENOR**
egistrado en la base de datos? Ordena de mayor a menor.
¿Cuál es el año con más películas en la base de datos
Obtén un listado de todos los géneros y cuántas películas corresponden a cada uno.
Muestra todas las películas cuyo título contenga la palabra "Godfather" (puedes usar cualquier
palabra).

In [266]:
query = "SELECT anio_lanzamiento, COUNT(*) AS cantidad_peliculas FROM movies GROUP BY anio_lanzamiento ORDER BY cantidad_peliculas DESC"
mycursor.execute(query)
resultados = mycursor.fetchall()

# Imprimir cada fila
print("Cantidad de películas por año (ordenadas de mayor a menor):")
for anio, cantidad in resultados:
    print(f"Año: {anio}, Número de películas: {cantidad}")





Cantidad de películas por año (ordenadas de mayor a menor):
Año: 2001, Número de películas: 75
Año: 2013, Número de películas: 60
Año: 1994, Número de películas: 60
Año: 2008, Número de películas: 60
Año: 1999, Número de películas: 60
Año: 2017, Número de películas: 60
Año: 2010, Número de películas: 45
Año: 1998, Número de películas: 45
Año: 2014, Número de películas: 45
Año: 2000, Número de películas: 45
Año: 1997, Número de películas: 45
Año: 2009, Número de películas: 45
Año: 2015, Número de películas: 45
Año: 1980, Número de películas: 45
Año: 2019, Número de películas: 45
Año: 2012, Número de películas: 45
Año: 2004, Número de películas: 45
Año: 2016, Número de películas: 45
Año: 2002, Número de películas: 45
Año: 2003, Número de películas: 45
Año: 2018, Número de películas: 45
Año: 1995, Número de películas: 30
Año: 2006, Número de películas: 30
Año: 1993, Número de películas: 30
Año: 1991, Número de películas: 30
Año: 1976, Número de películas: 30
Año: 1977, Número de películas